In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from bqskit.ext import qiskit_to_bqskit as converter_to_bqskit

qr = QuantumRegister(11, name='qr')
cr = ClassicalRegister(11, name='cr')
qc = QuantumCircuit(qr, cr, name='qc')
# TIMESTAMP: 1737634168.8635523

# Apply gate operations
# <START_GATES>
qc.cry(5.240754, qr[10], qr[6])
qc.rccx(qr[10], qr[9], qr[8])
qc.cu(1.022478, 3.041734, 0.120257, 1.923515, qr[3], qr[4])
qc.p(0.858157, qr[5])
qc.sxdg(qr[8])
# <END_GATES>


qc.draw()

qr_0: ────────────────────────────────────────────────────
                                                           
 qr_1: ────────────────────────────────────────────────────
                                                           
 qr_2: ────────────────────────────────────────────────────
                                                           
 qr_3: ─────────────────■──────────────────────────────────
       ┌────────────────┴────────────────┐                 
 qr_4: ┤ U(1.0225,3.0417,0.12026,1.9235) ├─────────────────
       └──────────┬────────────┬─────────┘                 
 qr_5: ───────────┤ P(0.85816) ├───────────────────────────
                  ├────────────┤                           
 qr_6: ───────────┤ Ry(5.2408) ├───────────────────────────
                  └─────┬──────┘                           
 qr_7: ─────────────────┼──────────────────────────────────
                        │                 ┌───────┐┌──────┐
 qr_8: ─────────────────┼─────────────────┤2      ├┤ √Xdg ├
                        │                 │       │└──────┘
 qr_9: ─────────────────┼─────────────────┤1 Rccx ├────────
                        │                 │       │        
qr_10: ─────────────────■─────────────────┤0      ├────────
                                          └───────┘        
cr: 11/════════════════════════════════════════════════════

In [3]:
qc_bqskit = converter_to_bqskit(qc)
print(qc_bqskit)

Circuit(11)[CUGate@(3, 4) ... Dagger(SqrtXGate)@(8,)]


In [6]:
from pathlib import Path
from typing import Optional


def export_to_qasm_with_bqskit(
        qiskit_circ: QuantumCircuit, var_name: str,
        abs_output_file: str = None) -> Optional[str]:
    """Export a Qiskit circuit to a BQSKit QASM file."""
    from bqskit.ext import qiskit_to_bqskit
    bqskit_circ = qiskit_to_bqskit(qiskit_circ)

    # Determine file path
    if not abs_output_file:
        current_file = Path(__file__)
        file_stem = current_file.stem
        file_path_bqskit = current_file.with_name(
            f"{file_stem}_{var_name}_bqskit.qasm")
    else:
        file_path_bqskit = Path(abs_output_file)

    # Save BQSKit circuit to QASM
    bqskit_circ.save(str(file_path_bqskit))

    print(f"Saved the BQSKit circuit to {file_path_bqskit}")
    return file_path_bqskit.as_posix()


export_to_qasm_with_bqskit(qc, "qc", "qc_bqskit.qasm")

AttributeError: 'DaggerGate' object has no attribute '_qasm_name'

In [ ]:
import pennylane as qml
from qiskit import QuantumCircuit
from qiskit.qasm2 import load
from pathlib import Path

target_file = "0001104_1b7227_qc_RemoveRedundancies_pytket.qasm"
target_file_content = Path(target_file).read_text()

# Load the QASM file
# qc_imported = load('0001104_1b7227_qc_RemoveRedundancies_pytket.qasm')

# qc_imported.draw()


# CORRECT

# from pytket.qasm import circuit_from_qasm_str

# tket_qc = circuit_from_qasm_str(target_file_content)

# from pytket.circuit.display import render_circuit_jupyter

# render_circuit_jupyter(tket_qc)

#

loaded_circuit = qml.from_qasm(target_file_content)

print(qml.draw(loaded_circuit)())

QASM2ParseError: "<input>:17,0: 'c4x' takes 5 quantum arguments, but got 4"

In [21]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

qr = QuantumRegister(4)
qc = QuantumCircuit(qr)
qc.mcx([qr[0], qr[1], qr[2]], qr[3])
qc.draw()

q2_0: ──■──
        │  
q2_1: ──■──
        │  
q2_2: ──■──
      ┌─┴─┐
q2_3: ┤ X ├
      └───┘

In [22]:
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.passes import RemoveRedundancies, PeepholeOptimise2Q
from pytket.qasm import circuit_to_qasm_str

# Convert Qiskit circuit to Pytket circuit
tkqc = qiskit_to_tk(qc)

# Apply Full Peephole Optimization
RemoveRedundancies().apply(tkqc)
# CRASH: RuntimeError: Operation is not a gate: Unitary1qBox

# Export optimized circuit as QASM
opt_circuit_qasm = circuit_to_qasm_str(tkqc, header='qelib1', maxwidth=200)
print(opt_circuit_qasm)

OPENQASM 2.0;
include "qelib1.inc";

qreg q2[4];
c4x q2[0],q2[1],q2[2],q2[3];



In [24]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.qasm import circuit_to_qasm_str

# Convert Qiskit circuit to Pytket circuit
tkqc = qiskit_to_tk(qc)

render_circuit_jupyter(tkqc)

# Export optimized circuit as QASM
exported_qasm = circuit_to_qasm_str(tkqc, header='qelib1', maxwidth=200)
print(exported_qasm)

OPENQASM 2.0;
include "qelib1.inc";

qreg q2[4];
c4x q2[0],q2[1],q2[2],q2[3];



In [25]:
from pytket.qasm import circuit_from_qasm_str
from pytket.circuit.display import render_circuit_jupyter

tket_qc = circuit_from_qasm_str(exported_qasm)
render_circuit_jupyter(tket_qc)

In [ ]:
from qiskit.qasm2 import loads
from qiskit import qasm2

qc_imported_qiskit = loads(
    exported_qasm, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
# QASM2ParseError: "<input>:5,0: 'c4x' takes 5 quantum arguments, but got 4"

QASM2ParseError: "<input>:5,0: 'c4x' takes 5 quantum arguments, but got 4"